<br></br>
# **나이브베이즈 모델의 활용**
1. **베이지안 평가 학습모델**을 불러온다 (0:부정 / 1:긍정)
1. **분석을 위한 네이버 뉴스 댓글**을 수집한다 (600개 내외가 적합)
1. **긍정/ 부정 댓글**을 분류한다
1. **긍정 리뷰**의 **Token** 을 트위터 **Stemming**으로 정규화 한다
1. **정규화된 Token** 중 **"명사", "동사", "형용사"** 만 수집
1. 수집된 Token List 객체를 **nltk.Text 로 변환 후 빈도상위 10개**를 출력한다

<img src="http://www.meconomynews.com/news/photo/201806/15255_14549_2116.png" />

<br></br>
## **1 모듈과 데이터 불러오기**
Import Data / Datum

In [4]:
# Quiz 1 : 앞에서 학습한 Classifier 모델을 불러온다
# >> './data/classifiers.model' --> classifier
import pickle
classifier = "=Quiz!="

In [3]:
# Quiz 2 : 분류기준 단어목록 불러오기
# >> './data/selected.words' --> selected_words
selected_words = "=Quiz!="

In [ ]:
# Tokenizer 생성/ 판단모듈
from konlpy.tag import Twitter
twitter = Twitter()

# 문단을 긍부정 분석용 Token 생성하기
def tokenize(doc):
    doc = ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

# 위에서 생성한 Token을 활용하여 판단하기
def classify_text(text):
    tokens = tokenize(text)             # 트위터로 Stemming/ Taggging 후 Selected_word 판단결과 객체
    return classifier.classify(tokens)  # 분류모델 평가

In [ ]:
# 데이터와 모델, 함수를 활용하여 긍/부정 Test
# 0 : 부정모델 , 1 : 긍정모델

text   = """영화가 재미있어요"""

# 학습 데이터를 활용하여 text 내용 긍/부정 판단
result = classify_text(text)
if result == '1': 
    print('긍정리뷰')
else:             
    print('부정리뷰')

<br></br>
## **2 뉴스 데이터 댓글데이터 불러오기**
replies [참고](http://blog.naver.com/PostView.nhn?blogId=sangdo14&logNo=220610607049&categoryNo=0&parentCategoryNo=163&viewDate=&currentPage=1&postListTopCurrentPage=1&from=search)

In [ ]:
# 외부뉴스 데이터 수집하기
from txtutil import Naver_news_rep
# news_url = "https://news.naver.com/main/hotissue/read.nhn?mid=hot&" +\
#            "sid1=&cid=1064721&iid=3227816&oid=001&aid=0010068060&ptype=021"
# replies  = Naver_news_rep(news_url)

# import pickle
# pickle.dump(replies, open('./data/naver_news.rep', 'wb'))

In [ ]:
# Quiz 3 : 네이버 뉴스 댓글 저장데이터 불러오기
# >> './data/naver_news.rep' --> data

import pickle
data = pickle.load(open('./data/naver_news.rep', 'rb'))
data[:5]

<br></br>
## **3 댓글자료를 긍정 부정 분류**
result['긍정'] / result['부정'] dict 객체로 저장

In [ ]:
# 전체 댓글 갯수중 긍정/ 부정 갯수 계산하기
pro, con = [], []
for datum in data:
    if classify_text(datum) == "1":
        pro.append(datum)
    else:
        con.append(datum)

In [ ]:
# 긍정 / 부정 분류된 데이터를 Json 과 같은 형태로 정리
result = {}
result["긍정"] = pro
result["부정"] = con

In [ ]:
# 긍정리뷰 갯수
len(result["긍정"])

In [ ]:
# 부정리뷰 갯수
len(result["부정"])

<br></br>
## **4 "부정분류"된 댓글의 자연어 분석**
result['긍정'] / result['부정'] dict 객체로 저장

In [ ]:
# 긍정 댓글을 구분하여 분류한 뒤
# 명사, 형용사, 동사 Token만 추출하기 (전처리 / Token의 예외처리는 미포함)
texts = result['부정']
texts[:3]

In [ ]:
%%time
filtered_tokens = ""

# 명사, 형용사, 동사 Filtering
for text in texts:
    for txt in text[0].split(' '):
        tokens = twitter.pos(text ,stem=True)
        tokens = [token[0]    for token in tokens    
                              if (token[1] in ['Noun','Adjective','Verb']) and (len(token[0]) > 1 )]
        tokens = " ".join(tokens)
        filtered_tokens += tokens
        
print(filtered_tokens[:300])

In [ ]:
from nltk.tokenize import word_tokenize
filtered_tokens = word_tokenize(filtered_tokens)

# nltk.Text() 객체를 활용
# 상위 빈도수 20인 Token을 추출
import nltk
pos_nltk = nltk.Text(filtered_tokens, name='부정댓글')
pos_nltk.vocab().most_common(20)